In [3]:
import io
import os
import json
from google.protobuf.json_format import MessageToJson
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pprint

In [4]:
from google.cloud import vision
from google.cloud.vision import types

In [5]:
client = vision.ImageAnnotatorClient()

DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://developers.google.com/accounts/docs/application-default-credentials.

In [ ]:
with io.open('./image_files/IMG_2053.jpg', 'rb') as image_file:
    content = image_file.read()

In [ ]:
image = types.Image(content=content)

In [ ]:
results = client.text_detection(image=image)
serialized = json.loads(MessageToJson(results))


In [ ]:
# blocks = serialized['fullTextAnnotation']['pages'][0]['blocks']
pp = pprint.PrettyPrinter(indent=4)
# pp.pprint(serialized['textAnnotations'])
blocks = serialized['textAnnotations']


text_boxes = []
for i in range(1, len(blocks)):
  block = blocks[i]
  text_box = {'text': block['description'], 'boundingPoly':[], 'midpoints':[], 'slope': None}
  for vertex in block['boundingPoly']['vertices']:
    curr_tuple = [vertex['x'], vertex['y']]
    text_box['boundingPoly'].append(curr_tuple)
  x = 0
  y = 1
  text_box['midpoints'].append(((text_box['boundingPoly'][0][x]+text_box['boundingPoly'][3][x])//2, 
                    (text_box['boundingPoly'][0][y]+text_box['boundingPoly'][3][y])//2 ))
  text_box['midpoints'].append(((text_box['boundingPoly'][1][x]+text_box['boundingPoly'][2][x])//2, 
                    (text_box['boundingPoly'][1][y]+text_box['boundingPoly'][2][y])//2 ))
  try:
      text_box['slope'] = (text_box['midpoints'][1][y]-text_box['midpoints'][0][y])/(text_box['midpoints'][1][x]-text_box['midpoints'][0][x])
  except ZeroDivisionError:
      text_box['slope'] = 1000000
  
        

  text_boxes.append(text_box)

pp.pprint(text_boxes)

In [ ]:
img = cv2.imread('./image_files/IMG_2053.jpg')
for text in text_boxes:
    bounds = text['boundingPoly']
    pts = np.array(bounds, np.int32)
    pts = pts.reshape((-1,1,2))
    img = cv2.polylines(img,[pts],True,(0,255,0),3)
    img = cv2.line(img,text['midpoints'][0],text['midpoints'][1],(255,0,0),5)
plt.rcParams['figure.figsize'] = (10,10)
plt.imshow(img)

In [ ]:
img = cv2.imread('./image_files/IMG_2053.jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)